# Create a dataset


In [15]:
corpus = "bn"
language = "bn"
dir_data = "/path/to/RESPIN_ASRU_Challenge_2023"
origin_file_path = [f"{dir_data}/corpus/{corpus}/train/text", f"{dir_data}/corpus/{corpus}/dev/text"]

In [16]:
import pandas as pd
from phonemizer import phonemize

### Read and combine data from all files

The data looks like this:

`sid` _ `txtid` _ `uttid` `text`

`16777288` _ `629046` _ `281474981563595` `"বাঁশের প্রায়"`


In [17]:
data = pd.concat([pd.read_csv(path, sep="\t", header=None) for path in origin_file_path], ignore_index=True)
print("Number of lines:", len(data))
data.head()

Number of lines: 581236


,0
0,16777288_629046_281474981563595 বাঁশের প্রায় দ...
1,16777288_629072_281474981405386 এক ধরনের পদ্ধত...
2,16777288_629108_281474981563619 গুগলি বা ঝিনুক...
3,16777288_629112_281474981581650 বাগ আরশোলা ইত্...
4,16777288_629121_281474981582236 প্রাকৃতিক সব জ...


### Convert the data to the format of sid txtid uttid text, where each value in own column


In [18]:
data = data[0].str.split("_", n=2, expand=True)

data = pd.concat([data[0], data[1], data[2].str.split(" ", n=1, expand=True)], axis=1)
data.columns = ["spkid", "txtid", "uttid", "text"]
data.head()

,spkid,txtid,uttid,text
0,16777288,629046,281474981563595,বাঁশের প্রায় দশ হাজার প্রজাতি হয় যেমন বাম্বুসা...
1,16777288,629072,281474981405386,এক ধরনের পদ্ধতি যেটা দিয়ে শস্য থেকে খোসা ছাড়ান...
2,16777288,629108,281474981563619,গুগলি বা ঝিনুকের মানে হল ওয়েস্টার আমরা খাই
3,16777288,629112,281474981581650,বাগ আরশোলা ইত্যাদি সব পোকা গুলোর চাষ হয়
4,16777288,629121,281474981582236,প্রাকৃতিক সব জিনিস গুলো দিয়া যখন চাষ করা হয়


### Map speaker ids to indices with sorted order


In [19]:
spkid_to_idx = {spkid: idx for idx, spkid in enumerate(sorted(data.spkid.unique()))}
spkidx = data["spkid"].map(spkid_to_idx)

spkidx = pd.DataFrame(spkidx)
spkidx.columns = ["spkidx"]
data = pd.concat([spkidx, data], axis=1)
data.head()

,spkidx,spkid,txtid,uttid,text
0,0,16777288,629046,281474981563595,বাঁশের প্রায় দশ হাজার প্রজাতি হয় যেমন বাম্বুসা...
1,0,16777288,629072,281474981405386,এক ধরনের পদ্ধতি যেটা দিয়ে শস্য থেকে খোসা ছাড়ান...
2,0,16777288,629108,281474981563619,গুগলি বা ঝিনুকের মানে হল ওয়েস্টার আমরা খাই
3,0,16777288,629112,281474981581650,বাগ আরশোলা ইত্যাদি সব পোকা গুলোর চাষ হয়
4,0,16777288,629121,281474981582236,প্রাকৃতিক সব জিনিস গুলো দিয়া যখন চাষ করা হয়


In [20]:
# Number of speakers
print("Number of speakers:", len(data.spkidx.unique()))

Number of speakers: 2011


### Convert the text to phonemes

It may take a while. F.e. 500_000 lines of text ~ 30 minutes


In [21]:
phonemes = phonemize(data["text"], backend="espeak", language=language, strip=True, preserve_punctuation=True, with_stress=True, tie=True, njobs=16)
data = data.assign(phonemized_text=phonemes)
data.head()

,spkidx,spkid,txtid,uttid,text,phonemized_text
0,0,16777288,629046,281474981563595,বাঁশের প্রায় দশ হাজার প্রজাতি হয় যেমন বাম্বুসা...,bˈãʃeɾ pɾˈajo dˈɔʃ hˈaɟaɾ pɾˈoɟatˌi hˈɔjo d͡ʒ...
1,0,16777288,629072,281474981405386,এক ধরনের পদ্ধতি যেটা দিয়ে শস্য থেকে খোসা ছাড়ান...,ˈek dʰˈɔɾɔnˌeɾ pˈɔddʰɔtˌi d͡ʒˈeʈa dˈie ʃˈɔssɔ ...
2,0,16777288,629108,281474981563619,গুগলি বা ঝিনুকের মানে হল ওয়েস্টার আমরা খাই,ɡˈuɡɔlˌi bˈa ɟʰˈinukˌeɾ mˈane hˈɔl ˈoːeʃʈˌaɾ ˌ...
3,0,16777288,629112,281474981581650,বাগ আরশোলা ইত্যাদি সব পোকা গুলোর চাষ হয়,bˈaɡ ˈaɾɔʃˌola ˈitædˌi ʃˈɔb pˈoka ɡˈuloɾ t͡ʃˈa...
4,0,16777288,629121,281474981582236,প্রাকৃতিক সব জিনিস গুলো দিয়া যখন চাষ করা হয়,pɾakɾˈitik ʃˈɔb ɟˈiniʃ ɡˈulo dˈia d͡ʒˈɔkʰɔn t͡...


In [22]:
print("Number of lines:", len(data))

Number of lines: 581236


### Save the data to .csv file


In [23]:
# data = data.iloc[:100]

data.to_csv(f"../filelists/metadata.csv", sep="|", index=False)